In [1]:
import pandas as pd
import numpy as np
import json
import csv
#1.确定这几个数据都是干什么的
swiping_card_dataframe = pd.read_csv('./bus_swiping_card_info1.csv')
card_info_dataframe = pd.read_csv('./card_info.csv')
bus_swip_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='刷卡数据',header=0,engine='openpyxl')
bus_station_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='站点数据',header=0,engine='openpyxl')
#swiping_card_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间
#card_info_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间也是上面的信息，但是时间是去年的。
#用去年的时间的同比数据，去预测今年的同比数据，看是否一致，如果一致，说明经济发展没有变化，如果变大，说明经济好转
#bus_swip_dataframe ：一天的统计信息：各条线路、各个站点的刷卡数量信息
#bus_station_dataframe ： 站点的统计信息：各个站点的经纬度

In [2]:
#从card_info_dataframe中找出所有用户一天内的od对信息
card_info_dataframe['custom_day'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[0])
card_info_dataframe['custom_precise_time'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[1])
#sum(card_info_dataframe['partitionday']==card_info_dataframe['custom_day'])
#card_info_dataframe
card_info_dataframe.drop(labels=['custom_time','card_type','consume','data_src','data_load_time','partitionday'],axis=1,inplace=True)

day_0929_dataframe = card_info_dataframe[card_info_dataframe['custom_day']=='2023-09-29']

#按照刷卡id进行聚合
every_user_all_trips_dataframe_0929 = \
day_0929_dataframe.groupby('card_identity').agg({'custom_precise_time':list, \
                                                 'line_identity':list, \
                                                 'station_identity':list, \
                                                'car_identity':list},axis=1).reset_index()
#每天承载18万人出行181190人
#乘坐两程以上的有109927人
#乘坐单程的极有可能是流动，也有可能是别的交通方式回。从前后相依有下一段旅程的来确定od对，对于没有后程的暂时不确定。
every_user_all_trips_dataframe_0929['trip_length'] = \
every_user_all_trips_dataframe_0929['line_identity'].apply(lambda x:len(x))

In [3]:
trip_great_than_2_0929= \
every_user_all_trips_dataframe_0929[ \
every_user_all_trips_dataframe_0929['trip_length']!=1].reset_index(drop=True)
trip_great_than_2_0929['trip'] = trip_great_than_2_0929.apply(lambda x:list(zip(x['custom_precise_time'],x['line_identity'],x['station_identity'])),axis=1)

trip_great_than_2_0929['ordered_trips'] = \
trip_great_than_2_0929['trip'].apply(lambda x:sorted(x,key=lambda x:x[0]))

In [4]:
#获取路网结构数据
#把每个车的刷卡数据按照时间排序，然后按照递增序列分组，
#然后把每组保留每个站点的第一个数据
#然后根据同样的时间间隔进行填充。
with open('./datas/road_line_station_structure_setting.json', 'r') as f:
    road_line_station_structure_setting = json.load(f)

station_distance_rank_index_list = np.loadtxt('./datas/station_pair_distance_rank_index.csv',delimiter=',',dtype=int)
# 从文件中读取字典
with open('./datas/id_2_station.json', 'r') as file:
    id_2_station = json.load(file)
    id_2_station = {int(key): value for key, value in id_2_station.items()}
# 将字典保存到文件中
with open('./datas/station_2_id.json', 'r') as file:
    station_2_id = json.load(file)

In [5]:
road_line_station_structure_setting

{'1': {'stations_nums': 20,
  'stations': {'1': '火车站',
   '2': '展览馆',
   '3': '新路口',
   '4': '邮电大楼',
   '5': '大十字',
   '6': '喷水池',
   '7': '云中广场',
   '8': '六广门',
   '9': '北京路',
   '10': '北京西路口',
   '11': '黔灵山公园',
   '12': '黔灵山公园',
   '13': '延安西路',
   '14': '紫林庵',
   '15': '大西门',
   '16': '次南门',
   '17': '河滨公园',
   '18': '新路口',
   '19': '展览馆',
   '20': '火车站'},
  'all_stations': ['火车站',
   '展览馆',
   '新路口',
   '邮电大楼',
   '大十字',
   '喷水池',
   '云中广场',
   '六广门',
   '北京路',
   '北京西路口',
   '黔灵山公园',
   '黔灵山公园',
   '延安西路',
   '紫林庵',
   '大西门',
   '次南门',
   '河滨公园',
   '新路口',
   '展览馆',
   '火车站']},
 '2': {'stations_nums': 20,
  'stations': {'1': '火车站',
   '2': '展览馆',
   '3': '新路口',
   '4': '河滨公园',
   '5': '次南门',
   '6': '大西门',
   '7': '紫林庵',
   '8': '延安西路',
   '9': '黔灵山公园',
   '10': '黔灵山公园',
   '11': '北京西路口',
   '12': '北京路',
   '13': '六广门',
   '14': '云中广场',
   '15': '喷水池',
   '16': '大十字',
   '17': '邮电大楼',
   '18': '新路口',
   '19': '展览馆',
   '20': '火车站'},
  'all_stations': ['火车站',
   '展览馆',
   '新路口',
  

In [6]:
#根据路网结构构件一个字典:{line_id:{order_id:station_name}}
#{station_id:{'station_name':station_name,lines_orders:{line:[order_id]}}}
station_2_lines_order = {}
for line_id in road_line_station_structure_setting.keys():
    line_stations = road_line_station_structure_setting[line_id]['stations']
    for station_id,station_name in line_stations.items():
        station_global_id = station_2_id[station_name]
        if station_global_id not in station_2_lines_order:
            station_2_lines_order[station_global_id] = {}
            station_2_lines_order[station_global_id][int(line_id)]=[int(station_id)]
        else:
            if int(line_id) not in station_2_lines_order[station_global_id]:
                station_2_lines_order[station_global_id][int(line_id)] = [int(station_id)]
            else:
                station_2_lines_order[station_global_id][int(line_id)].append(int(station_id))
station_2_lines_order


{806: {1: [1, 20],
  2: [1, 20],
  17: [1, 31],
  18: [20, 21],
  24: [1, 34],
  43: [30, 31],
  60: [26, 27],
  63: [16, 17],
  65: [28, 29],
  74: [17, 18],
  78: [17, 18],
  203: [1, 82],
  205: [1, 67],
  216: [1, 13],
  219: [18, 19],
  224: [23, 24],
  237: [43, 44],
  240: [8, 9],
  250: [24, 25],
  253: [27, 28],
  255: [25, 26],
  272: [14, 15],
  280: [24, 25],
  1012: [16, 17],
  1018: [9, 10],
  1019: [35, 36],
  1024: [28, 29],
  1055: [5, 6],
  1101: [1, 12],
  1203: [1, 4],
  4002: [2, 3],
  4130: [1, 18],
  4330: [17, 18],
  4510: [15, 16],
  6004: [27, 28],
  6006: [35, 36],
  6008: [28, 29],
  6009: [17, 18],
  6013: [14, 15],
  18224: [22, 23]},
 508: {1: [2, 19],
  2: [2, 19],
  6: [12, 23],
  17: [2, 30],
  18: [19, 22],
  24: [2, 33],
  29: [33, 38],
  65: [27, 30],
  78: [16, 19],
  237: [42, 45],
  253: [26, 29],
  272: [13, 16],
  308: [10, 15],
  1012: [15, 18],
  1014: [17, 29],
  1018: [8, 11],
  1024: [27, 30],
  4300: [14],
  6003: [19, 32],
  6004: [26, 2

# 要想确定出行路径，就势必要把每个用户的刷卡数据前后接起来

In [1]:
def find_leave_station(sorted_index_array:np.ndarray,specified_indices:list):
    # 找到指定索引组中在排序数组中的位置
    positions = [sorted_index_array.index(idx) for idx in specified_indices]
    return station_2_lines_order[sorted_index_array[min(positions)]]
def find_closest(lst, target):
    closest_value = min(lst, key=lambda x: abs(x - target))
    return closest_value
def get_od_pair(x):
    od_trips=[]
    for i in range(len(x)-1):
        this_time,this_line,this_station_id = x[i]
        next_time,next_line,next_station_id = x[i+1]
        try:
            #print(this_time,this_line,this_station_id)
            #print(next_time,next_line,next_station_id)
            this_line_station_name = road_line_station_structure_setting[str(this_line)]['stations'][str(this_station_id)]
            this_line_station_id = station_2_id[this_line_station_name]
            #print(i,this_line_station_name,this_line_station_id)
            all_possible_leave_stations = road_line_station_structure_setting[str(this_line)]['all_stations']
            all_possible_leave_stations_index = [station_2_id[station] for station in all_possible_leave_stations]
            #print(all_possible_leave_stations_index)
            next_line_station_name = road_line_station_structure_setting[str(next_line)]['stations'][str(next_station_id)]
            next_line_station_id = station_2_id[next_line_station_name]
            #print(next_line_station_id)
            distance_2_next_line_station_ndarray  = station_distance_rank_index_list[next_line_station_id]
            #leave_station = find_leave_station(distance_2_next_line_station_ndarray.tolist(),all_possible_leave_stations_index)
            leave_station_dict = find_leave_station(distance_2_next_line_station_ndarray.tolist(),all_possible_leave_stations_index)
            leave_station = find_closest(leave_station_dict[this_line], this_line_station_id)
            #print(this_line_station_id,leave_station,this_time,this_line)
            od_trip = {'this_line_station_id':this_station_id, \
                       'leave_station':leave_station,\
                       'this_time':this_time,'this_line':this_line}
            #print(od_trip)
            od_trips.append(od_trip)
        except:
            True
    return od_trips

NameError: name 'np' is not defined

In [20]:
trip_great_than_2_0929['od_trip_time_line']= \
trip_great_than_2_0929['ordered_trips'].apply(lambda x:get_od_pair(x))
trip_great_than_2_0929
trip_great_than_2_0929_exploded = trip_great_than_2_0929.explode('od_trip_time_line').reset_index(drop=True)
trip_great_than_2_0929_exploded

,card_identity,custom_precise_time,line_identity,station_identity,car_identity,trip_length,trip,ordered_trips,od_trip_time_line
0,0x0000719AE3ECECBABAD0263AF940450F,"[19:01:59, 16:55:43]","[40, 46]","[31, 1]","[4483, 3762]",2,"[(19:01:59, 40, 31), (16:55:43, 46, 1)]","[(16:55:43, 46, 1), (19:01:59, 40, 31)]","{'this_line_station_id': 1, 'leave_station': 2..."
1,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16...","[(09:20:51, 204, 8), (09:26:05, 207, 14), (16:...","{'this_line_station_id': 8, 'leave_station': 6..."
2,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16...","[(09:20:51, 204, 8), (09:26:05, 207, 14), (16:...","{'this_line_station_id': 14, 'leave_station': ..."
3,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16...","[(09:20:51, 204, 8), (09:26:05, 207, 14), (16:...","{'this_line_station_id': 5, 'leave_station': 2..."
4,0x000229AC214F69A81F266A521394C628,"[07:13:30, 18:07:56]","[7, 19]","[21, 23]","[5020, 5038]",2,"[(07:13:30, 7, 21), (18:07:56, 19, 23)]","[(07:13:30, 7, 21), (18:07:56, 19, 23)]","{'this_line_station_id': 21, 'leave_station': ..."
...,...,...,...,...,...,...,...,...,...
181420,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0...","[(15:07:03, 229, 34), (15:07:13, 229, 34), (16...","{'this_line_station_id': 34, 'leave_station': ..."
181421,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0...","[(15:07:03, 229, 34), (15:07:13, 229, 34), (16...","{'this_line_station_id': 34, 'leave_station': ..."
181422,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0...","[(15:07:03, 229, 34), (15:07:13, 229, 34), (16...","{'this_line_station_id': 14, 'leave_station': ..."
181423,0xFFFDF917CDD67471BC901AD187061D38,"[14:03:34, 13:16:22]","[47, 60]","[2, 6]","[5042, 4484]",2,"[(14:03:34, 47, 2), (13:16:22, 60, 6)]","[(13:16:22, 60, 6), (14:03:34, 47, 2)]","{'this_line_station_id': 6, 'leave_station': 2..."


In [21]:
def get_start_station(x):
    if not isinstance(x,float):
        return x['this_line_station_id']
    else:
        return -1
def get_leave_station(x):
    if not isinstance(x,float):
        return x['leave_station']
    else:
        return -1
def get_this_time(x):
    if not isinstance(x,float):
        return x['this_time']
    else:
        return -1
def get_this_line(x):
    if not isinstance(x,float):
        return x['this_line']
    else:
        return -1
trip_great_than_2_0929_exploded['this_od_trip_start_station']= \
trip_great_than_2_0929_exploded['od_trip_time_line'].apply(lambda x:get_start_station(x))
trip_great_than_2_0929_exploded['this_od_end_station']= \
trip_great_than_2_0929_exploded['od_trip_time_line'].apply(lambda x:get_leave_station(x))
trip_great_than_2_0929_exploded['this_od_start_time']= \
trip_great_than_2_0929_exploded['od_trip_time_line'].apply(lambda x:get_this_time(x))
trip_great_than_2_0929_exploded['this_od_by_line']= \
trip_great_than_2_0929_exploded['od_trip_time_line'].apply(lambda x:get_this_line(x))
trip_great_than_2_0929_exploded


,card_identity,custom_precise_time,line_identity,station_identity,car_identity,trip_length,trip,ordered_trips,od_trip_time_line,this_od_trip_start_station,this_od_end_station,this_od_start_time,this_od_by_line
0,0x0000719AE3ECECBABAD0263AF940450F,"[19:01:59, 16:55:43]","[40, 46]","[31, 1]","[4483, 3762]",2,"[(19:01:59, 40, 31), (16:55:43, 46, 1)]","[(16:55:43, 46, 1), (19:01:59, 40, 31)]","{'this_line_station_id': 1, 'leave_station': 2...",1,24,16:55:43,46
1,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16...","[(09:20:51, 204, 8), (09:26:05, 207, 14), (16:...","{'this_line_station_id': 8, 'leave_station': 6...",8,67,09:20:51,204
2,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16...","[(09:20:51, 204, 8), (09:26:05, 207, 14), (16:...","{'this_line_station_id': 14, 'leave_station': ...",14,30,09:26:05,207
3,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16...","[(09:20:51, 204, 8), (09:26:05, 207, 14), (16:...","{'this_line_station_id': 5, 'leave_station': 2...",5,27,16:29:50,5
4,0x000229AC214F69A81F266A521394C628,"[07:13:30, 18:07:56]","[7, 19]","[21, 23]","[5020, 5038]",2,"[(07:13:30, 7, 21), (18:07:56, 19, 23)]","[(07:13:30, 7, 21), (18:07:56, 19, 23)]","{'this_line_station_id': 21, 'leave_station': ...",21,12,07:13:30,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
181420,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0...","[(15:07:03, 229, 34), (15:07:13, 229, 34), (16...","{'this_line_station_id': 34, 'leave_station': ...",34,34,15:07:03,229
181421,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0...","[(15:07:03, 229, 34), (15:07:13, 229, 34), (16...","{'this_line_station_id': 34, 'leave_station': ...",34,35,15:07:13,229
181422,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0...","[(15:07:03, 229, 34), (15:07:13, 229, 34), (16...","{'this_line_station_id': 14, 'leave_station': ...",14,14,16:00:55,12
181423,0xFFFDF917CDD67471BC901AD187061D38,"[14:03:34, 13:16:22]","[47, 60]","[2, 6]","[5042, 4484]",2,"[(14:03:34, 47, 2), (13:16:22, 60, 6)]","[(13:16:22, 60, 6), (14:03:34, 47, 2)]","{'this_line_station_id': 6, 'leave_station': 2...",6,27,13:16:22,60


In [24]:
#从上面的数据中看看有没有逆向的？
trip_great_than_2_0929_exploded['forward'] = \
trip_great_than_2_0929_exploded.\
    apply(lambda x:x['this_od_trip_start_station']<=x['this_od_end_station'],axis=1)
trip_great_than_2_0929_exploded

,card_identity,custom_precise_time,line_identity,station_identity,car_identity,trip_length,trip,ordered_trips,od_trip_time_line,this_od_trip_start_station,this_od_end_station,this_od_start_time,this_od_by_line,forward
0,0x0000719AE3ECECBABAD0263AF940450F,"[19:01:59, 16:55:43]","[40, 46]","[31, 1]","[4483, 3762]",2,"[(19:01:59, 40, 31), (16:55:43, 46, 1)]","[(16:55:43, 46, 1), (19:01:59, 40, 31)]","{'this_line_station_id': 1, 'leave_station': 2...",1,24,16:55:43,46,True
1,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16...","[(09:20:51, 204, 8), (09:26:05, 207, 14), (16:...","{'this_line_station_id': 8, 'leave_station': 6...",8,67,09:20:51,204,True
2,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16...","[(09:20:51, 204, 8), (09:26:05, 207, 14), (16:...","{'this_line_station_id': 14, 'leave_station': ...",14,30,09:26:05,207,True
3,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16...","[(09:20:51, 204, 8), (09:26:05, 207, 14), (16:...","{'this_line_station_id': 5, 'leave_station': 2...",5,27,16:29:50,5,True
4,0x000229AC214F69A81F266A521394C628,"[07:13:30, 18:07:56]","[7, 19]","[21, 23]","[5020, 5038]",2,"[(07:13:30, 7, 21), (18:07:56, 19, 23)]","[(07:13:30, 7, 21), (18:07:56, 19, 23)]","{'this_line_station_id': 21, 'leave_station': ...",21,12,07:13:30,7,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181420,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0...","[(15:07:03, 229, 34), (15:07:13, 229, 34), (16...","{'this_line_station_id': 34, 'leave_station': ...",34,34,15:07:03,229,True
181421,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0...","[(15:07:03, 229, 34), (15:07:13, 229, 34), (16...","{'this_line_station_id': 34, 'leave_station': ...",34,35,15:07:13,229,True
181422,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0...","[(15:07:03, 229, 34), (15:07:13, 229, 34), (16...","{'this_line_station_id': 14, 'leave_station': ...",14,14,16:00:55,12,True
181423,0xFFFDF917CDD67471BC901AD187061D38,"[14:03:34, 13:16:22]","[47, 60]","[2, 6]","[5042, 4484]",2,"[(14:03:34, 47, 2), (13:16:22, 60, 6)]","[(13:16:22, 60, 6), (14:03:34, 47, 2)]","{'this_line_station_id': 6, 'leave_station': 2...",6,27,13:16:22,60,True


In [28]:
trip_great_than_2_0929_exploded['forward'].sum()
trip_great_than_2_0929_exploded = trip_great_than_2_0929_exploded[trip_great_than_2_0929_exploded['forward']].reset_index(drop=True)
trip_great_than_2_0929_exploded

,card_identity,custom_precise_time,line_identity,station_identity,car_identity,trip_length,trip,ordered_trips,od_trip_time_line,this_od_trip_start_station,this_od_end_station,this_od_start_time,this_od_by_line,forward
0,0x0000719AE3ECECBABAD0263AF940450F,"[19:01:59, 16:55:43]","[40, 46]","[31, 1]","[4483, 3762]",2,"[(19:01:59, 40, 31), (16:55:43, 46, 1)]","[(16:55:43, 46, 1), (19:01:59, 40, 31)]","{'this_line_station_id': 1, 'leave_station': 2...",1,24,16:55:43,46,True
1,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16...","[(09:20:51, 204, 8), (09:26:05, 207, 14), (16:...","{'this_line_station_id': 8, 'leave_station': 6...",8,67,09:20:51,204,True
2,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16...","[(09:20:51, 204, 8), (09:26:05, 207, 14), (16:...","{'this_line_station_id': 14, 'leave_station': ...",14,30,09:26:05,207,True
3,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16...","[(09:20:51, 204, 8), (09:26:05, 207, 14), (16:...","{'this_line_station_id': 5, 'leave_station': 2...",5,27,16:29:50,5,True
4,0x00065B8345E25E499BDF9B0632576C87,"[16:26:36, 09:58:48]","[57, 57]","[6, 32]","[7344, 7343]",2,"[(16:26:36, 57, 6), (09:58:48, 57, 32)]","[(09:58:48, 57, 32), (16:26:36, 57, 6)]","{'this_line_station_id': 32, 'leave_station': ...",32,43,09:58:48,57,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168294,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0...","[(15:07:03, 229, 34), (15:07:13, 229, 34), (16...","{'this_line_station_id': 34, 'leave_station': ...",34,34,15:07:03,229,True
168295,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0...","[(15:07:03, 229, 34), (15:07:13, 229, 34), (16...","{'this_line_station_id': 34, 'leave_station': ...",34,35,15:07:13,229,True
168296,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0...","[(15:07:03, 229, 34), (15:07:13, 229, 34), (16...","{'this_line_station_id': 14, 'leave_station': ...",14,14,16:00:55,12,True
168297,0xFFFDF917CDD67471BC901AD187061D38,"[14:03:34, 13:16:22]","[47, 60]","[2, 6]","[5042, 4484]",2,"[(14:03:34, 47, 2), (13:16:22, 60, 6)]","[(13:16:22, 60, 6), (14:03:34, 47, 2)]","{'this_line_station_id': 6, 'leave_station': 2...",6,27,13:16:22,60,True


In [29]:
#this_od_trip_start_station	this_od_end_station	this_od_start_time	this_od_by_line
need_attributes = ['card_identity','this_od_trip_start_station','this_od_end_station','this_od_start_time','this_od_by_line']
all_known_od_trips_0929 = trip_great_than_2_0929_exploded[need_attributes]

can_use_all_known_od_trips_0929 = all_known_od_trips_0929[all_known_od_trips_0929['this_od_trip_start_station']!=-1].reset_index(drop=True)

can_use_all_known_od_trips_0929['hour'] = can_use_all_known_od_trips_0929['this_od_start_time'].apply(lambda x:int(x.split(':')[0]))
can_use_all_known_od_trips_0929['min'] = can_use_all_known_od_trips_0929['this_od_start_time'].apply(lambda x:int(x.split(':')[1]))
can_use_all_known_od_trips_0929['second'] = can_use_all_known_od_trips_0929['this_od_start_time'].apply(lambda x:int(x.split(':')[2]))
can_use_all_known_od_trips_0929

#接下来怎么从已有的这些数据中，确定每个站点在各个时间段走向各个方向的概率？同一时间段上车的人在哪里下车的概率。

,card_identity,this_od_trip_start_station,this_od_end_station,this_od_start_time,this_od_by_line,hour,min,second
0,0x0000719AE3ECECBABAD0263AF940450F,1,24,16:55:43,46,16,55,43
1,0x0000D15C7814D19BE81C40B3ED9CA55C,8,67,09:20:51,204,9,20,51
2,0x0000D15C7814D19BE81C40B3ED9CA55C,14,30,09:26:05,207,9,26,5
3,0x0000D15C7814D19BE81C40B3ED9CA55C,5,27,16:29:50,5,16,29,50
4,0x00065B8345E25E499BDF9B0632576C87,32,43,09:58:48,57,9,58,48
...,...,...,...,...,...,...,...,...
155968,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,34,34,15:07:03,229,15,7,3
155969,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,34,35,15:07:13,229,15,7,13
155970,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,14,14,16:00:55,12,16,0,55
155971,0xFFFDF917CDD67471BC901AD187061D38,6,27,13:16:22,60,13,16,22


In [12]:
#时间精度 minutes
time_accurate = 8
time_split_array = list(range(0,60+time_accurate,time_accurate))
labels = list(range(len(time_split_array)-1))
can_use_all_known_od_trips_0929['minute_group'] = pd.cut(can_use_all_known_od_trips_0929['min'], bins=time_split_array, right=False, labels=labels)
different_time_user_produce = \
    can_use_all_known_od_trips_0929.\
        groupby(['this_od_trip_start_station','hour','minute_group'],observed=False)\
            .agg({'this_od_end_station':list,'this_od_by_line':list}).reset_index()
different_time_user_produce

,this_od_trip_start_station,hour,minute_group,this_od_end_station,this_od_by_line
0,0,5,0,NaN,NaN
1,0,5,1,NaN,NaN
2,0,5,2,NaN,NaN
3,0,5,3,NaN,NaN
4,0,5,4,NaN,NaN
...,...,...,...,...,...
171907,1448,23,3,NaN,NaN
171908,1448,23,4,NaN,NaN
171909,1448,23,5,NaN,NaN
171910,1448,23,6,NaN,NaN


In [14]:
#如果我保持上面的数据
#然后根据站点和时间和时段使用merge来获取this_od_end_station和this_od_by_line
#应该也是可以的。
#当前时刻
now_hour = 8
now_minute = 23
#更新全部站点的情况，dataframe为站点列表，时间，minute
station_id_list = list(id_2_station.keys())
now_conditions_df=pd.DataFrame({'this_od_trip_start_station':station_id_list,\
                            'hour':[now_hour]*len(station_id_list),\
                            'minute_group':[now_minute//time_accurate]*len(station_id_list)})
#使用merge进行合并更新
now_user_all_possible_route_df = now_conditions_df.merge(different_time_user_produce, on=['this_od_trip_start_station', 'hour', 'minute_group'], how='left').fillna(0)

def zip_target_station_line_func(x):
    if isinstance(x['this_od_end_station'],list):
        return list(zip(x['this_od_end_station'],x['this_od_by_line']))
    else:
        return []
now_user_all_possible_route_df['zip_target_station_line'] = now_user_all_possible_route_df.apply(lambda x:zip_target_station_line_func(x),axis=1)
now_user_all_possible_route_df

,this_od_trip_start_station,hour,minute_group,this_od_end_station,this_od_by_line,zip_target_station_line
0,0,8,2,0,0,[]
1,1,8,2,0,0,[]
2,2,8,2,"[17, 239, 7, 396, 627]","[253, 234, 253, 56, 56]","[(17, 253), (239, 234), (7, 253), (396, 56), (..."
3,3,8,2,0,0,[]
4,4,8,2,0,0,[]
...,...,...,...,...,...,...
1444,1444,8,2,0,0,[]
1445,1445,8,2,0,0,[]
1446,1446,8,2,0,0,[]
1447,1447,8,2,0,0,[]


In [17]:
#将不同时刻出行人的数量也拼接起来。
#因为之前只用了两程，现在必须要根据刷卡数据确定各个站点的上车人数
def get_station_id(line_id,order_id):
    try:
        return station_2_id[road_line_station_structure_setting[str(line_id)]['stations'][str(order_id)]]
    except:
        return -1
day_0929_dataframe['this_od_trip_start_station'] = day_0929_dataframe.apply(lambda x: get_station_id(x['line_identity'],x['station_identity']),axis=1)
can_use_user_data = day_0929_dataframe[day_0929_dataframe['this_od_trip_start_station']!=-1].reset_index(drop=True)[['custom_precise_time','this_od_trip_start_station']]
can_use_user_data['hour'] = can_use_user_data.apply(lambda x:int(x['custom_precise_time'].split(':')[0]),axis=1)
can_use_user_data['minute'] = can_use_user_data.apply(lambda x:int(x['custom_precise_time'].split(':')[1]),axis=1)
can_use_user_data['seconds'] = can_use_user_data.apply(lambda x:int(x['custom_precise_time'].split(':')[2]),axis=1)
can_use_user_data
#custom_precise_time	this_od_trip_start_station	hour	minute	seconds
#this_od_trip_start_station	hour	minute_group	this_od_end_station	this_od_by_line	zip_target_station_line


/tmp/ipykernel_19914/1827131317.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_0929_dataframe['this_od_trip_start_station'] = day_0929_dataframe.apply(lambda x: get_station_id(x['line_identity'],x['station_identity']),axis=1)


,custom_precise_time,this_od_trip_start_station,hour,minute,seconds
0,08:59:40,1255,8,59,40
1,11:08:22,516,11,8,22
2,08:51:28,1140,8,51,28
3,22:32:48,1257,22,32,48
4,19:50:37,933,19,50,37
...,...,...,...,...,...
338281,08:48:38,759,8,48,38
338282,10:33:45,349,10,33,45
338283,14:57:10,249,14,57,10
338284,15:08:49,1058,15,8,49


In [18]:
#时间精度 minutes
time_accurate = 8
time_split_array = list(range(0,60+time_accurate,time_accurate))
labels = list(range(len(time_split_array)-1))
can_use_user_data['minute_group'] = pd.cut(can_use_user_data['minute'], bins=time_split_array, right=False, labels=labels)
different_time_user_produce_nums = can_use_user_data.groupby(['this_od_trip_start_station','hour','minute_group'],observed=False).size().reset_index(name='count')
different_time_user_produce_nums
#this_od_trip_start_station	hour	minute_group	count
#this_od_trip_start_station	hour	minute_group	this_od_end_station	this_od_by_line	zip_target_station_line

,this_od_trip_start_station,hour,minute_group,count
0,0,5,0,0
1,0,5,1,0
2,0,5,2,0
3,0,5,3,0
4,0,5,4,0
...,...,...,...,...
177531,1448,23,3,0
177532,1448,23,4,0
177533,1448,23,5,0
177534,1448,23,6,0


In [19]:
#把人数与可用的路径合并
#使用merge进行合并更新
now_station_routes_and_nums = \
    now_user_all_possible_route_df.merge(different_time_user_produce_nums,\
                                     on=['this_od_trip_start_station', 'hour', 'minute_group'],\
                                     how='left').fillna(0)

/tmp/ipykernel_19914/3380220038.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  how='left').fillna(0)


True

In [21]:
import random
def uniform_sample(x):
    if len(x['zip_target_station_line'])!=0:
        return random.choices(x['zip_target_station_line'],k=int(x['count']))
    else:
        return []

now_station_routes_and_nums['users'] = now_station_routes_and_nums.apply(lambda x:uniform_sample(x),axis=1)
now_station_routes_and_nums

,this_od_trip_start_station,hour,minute_group,this_od_end_station,this_od_by_line,zip_target_station_line,count,users
0,0,8,2,0,0,[],0.0,[]
1,1,8,2,0,0,[],0.0,[]
2,2,8,2,"[17, 239, 7, 396, 627]","[253, 234, 253, 56, 56]","[(17, 253), (239, 234), (7, 253), (396, 56), (...",6.0,"[(239, 234), (239, 234), (239, 234), (396, 56)..."
3,3,8,2,0,0,[],0.0,[]
4,4,8,2,0,0,[],0.0,[]
...,...,...,...,...,...,...,...,...
1444,1444,8,2,0,0,[],0.0,[]
1445,1445,8,2,0,0,[],0.0,[]
1446,1446,8,2,0,0,[],0.0,[]
1447,1447,8,2,0,0,[],0.0,[]


In [23]:
#从上面可以知道各个时刻各个站点有多少人往什么方向走。
#然后我们需要吧乘客的出行时间和目的地以及线路记录下来。
all_passengers = []
for row_index in range(len(now_station_routes_and_nums)):
    if len(now_station_routes_and_nums[row_index:row_index+1]['users'].values[0])!=0:
        #print(merged_df[row_index:row_index+1]['users'].values[0])
        start_station = now_station_routes_and_nums[row_index:row_index+1]['this_od_trip_start_station'].values[0]
        start_time = str(now_hour)+':'+str(now_minute)+':00'
        #start_time = now_station_routes_and_nums[row_index:row_index+1]['minute_group'].values[0]
        for user_instance in now_station_routes_and_nums[row_index:row_index+1]['users'].values[0]:
            #print(user_instance)
            target_station = user_instance[0]
            target_line = user_instance[1]
            all_passengers.append({\
                'start_station':start_station,\
                'start_time':start_time,\
                'target_station':target_station,\
                'target_line':target_line})
all_passengers

[{'start_station': 2,
  'start_time': '8:23:00',
  'target_station': 239,
  'target_line': 234},
 {'start_station': 2,
  'start_time': '8:23:00',
  'target_station': 239,
  'target_line': 234},
 {'start_station': 2,
  'start_time': '8:23:00',
  'target_station': 239,
  'target_line': 234},
 {'start_station': 2,
  'start_time': '8:23:00',
  'target_station': 396,
  'target_line': 56},
 {'start_station': 2,
  'start_time': '8:23:00',
  'target_station': 627,
  'target_line': 56},
 {'start_station': 2,
  'start_time': '8:23:00',
  'target_station': 17,
  'target_line': 253},
 {'start_station': 7,
  'start_time': '8:23:00',
  'target_station': 239,
  'target_line': 253},
 {'start_station': 7,
  'start_time': '8:23:00',
  'target_station': 806,
  'target_line': 253},
 {'start_station': 7,
  'start_time': '8:23:00',
  'target_station': 508,
  'target_line': 253},
 {'start_station': 7,
  'start_time': '8:23:00',
  'target_station': 534,
  'target_line': 253},
 {'start_station': 7,
  'start_ti

In [ ]:
#接下来要把公交运行时刻表引入进来。
#当前乘客的数据结构为：
#{  'start_station': 398,
#   'start_time': '8:23:00',
#   'target_station': 263,
#   'target_line': 806}
# 首先从线路找到可用的公交车
# 然后从可用的公交车找到线路方向符合要求的东西
# 已知线路到公交车的字典为：
# line_2_buses[line_identity] = car_identity
# 已知公交车的运行时间表为：
# arrive_time_2_station_id[key]
# 合并以后为：line_2_time_df[line_id]
#对于每个用户来说，检查 在这个时间表中， 他的目的地在不在当前站点的后面，
#如果在当前站点的后面，那么就可以把到站时间赋值给他。
#时间表存在问题
total_passengers = []
this_all_passengers = all_passengers
for user in this_all_passengers:
    timetable = line_2_time_df[line_id]
    this_time = user['start_time']
    this_station = user['start_station']
    index = timetable[this_station].index(user['start_time'])
    user['time_table'] = timetable[index][this_station:]
    total_passengers.append(user)

In [ ]:
#获取各路段的乘客数量。
road_section_2_nums = {}
for user in total_passengers:
    if this_time > user['time_table'][-1]:
        continue
    else:
        index = user['time_table'].index(this_time)
        this_station = user['time_table'][index]
        next_station = user['time_table'][index+1]
        road_section_2_nums[f"{this_station}_{next_station}"] += 1


```yaml
起点站:
起始时间:
终点站:
线路:
时刻表:
    到站时间: []
    到达站: []
```

In [1]:
#结合之前获得的公交车的运行时刻表
#得到了乘客以后，顺手就把这个乘客在各个时刻的位置表示出来吧。
#按照就近原则，分配一个公交车，把公交车的时刻表复制给这个节点。
#如果我们获得了这个时刻表，那么我们在给定的时间下，遍历所有乘客，
#如果当前时间超过了这个乘客到达终点站的时间，那么这个乘客不用考虑了
#在当前时间不超过终点站时间的时候，乘客的位置就根据索引，确定在什么站里面。
#对所有乘客进行遍历，如果当前时间不超过乘客的终止时间，那么对应位置路段的乘客数量加一。
#然后我们可以对路段进行排序。找出人数最多的路段，然后按照人数的多少画成网络图

      线路编号  方向  站点编号   站点名称          日期  刷卡数量
0        1   1     1    火车站  2023-10-01    17
1        1   1     2    展览馆  2023-10-01   318
2        1   1     3    新路口  2023-10-01   510
3        1   1     4   邮电大楼  2023-10-01   678
4        1   1     5    大十字  2023-10-01   598
...    ...  ..   ...    ...         ...   ...
6475  4005   0    10   南垭路口  2023-10-01     2
6476  4600   1     7    粑粑坳  2023-10-01     2
6477  4601   1    14  白金大道中  2023-10-01     1
6478  4601   1    16    刚玉街  2023-10-01     1
6479  4601   1    18  云环东路口  2023-10-01     1

[6480 rows x 6 columns]


```yaml
公交站id:
  时刻1: 
    可选路线: []
    对应路线的概率: []
    各路线的站点情况:
      路线id:
        本线路可达站点列表: []
        本线路可达站点概率: []
  时刻2: 
    可选路线: []
    对应路线的概率: []
    各路线的站点情况:
      路线id:
        本线路可达站点列表: []
        本线路可达站点概率: []
```